# Programming Metodologies for Data Analysis

Lorenzo Dell'Oro<br>Giovanni Toto<br>Gian Luca Vriz

## 1. Introduction

<font color='red'>KEY IDEA AND OBJECTIVE OF THE PROJECT</font>

## 2. Pre-processing of the corpus

<font color='blue'>BUILDING INPUTS FOR THE FUNCTIONS THAT ESTIMATE THE MODELS</font>

## 3. Explorative analysis of the processed corpus

<font color='blue'>EXPLORATORY ANALYSIS OF THE CORPUS</font>

## 4. Estimation of the topic models

<font color='red'>INTRODUCTION TO MODEL ESTIMATION</font>

### 4.i. Latent Dirichlet Allocation (LDA)

<font color='red'>BRIEF DESCRIPTION OF THE MODEL</font><br>
<font color='blue'>MODEL ESTIMATION</font><br>
*DO THE SAME FOR ALL TOPIC MODELS*

### 4.ii. Dynamic Topic Model (DTM)

### 4.iii. Embedded Topic Model (ETM)

### 4.iv. Dynamic Embedded Topic Model (DETM)

## 5. Model comparison

<font color='red'>INTRODUCTION ON HOW WE WANT TO COMPARE MODELS</font>

### 5.i. Quantitative analysis

<font color='blue'>COMPUTATION OF VARIOUS METRICS AND CONSTRUCTION OF GRAPHS</font>

### 5.ii. Qualitative analysis

<font color='blue'>INTERPRETATION OF TOPICS AND DOCUMENT REPRESENTATION (LDA vs ETM, DTM vs DETM)</font>

## 6. Conclusion

<font color='red'>FINAL REMARKS</font>

## References

Blei, D. M., Ng, A. Y., & Jordan, M. I. (2003). Latent dirichlet allocation. Journal of machine Learning research, 3(Jan), 993-1022. [ACM Digital Library](https://dl.acm.org/doi/10.5555/944919.944937)

Blei, D. M., & Lafferty, J. D. (2006, June). Dynamic topic models. In Proceedings of the 23rd international conference on Machine learning (pp. 113-120). [ACM Digital Library](https://doi.org/10.1145/1143844.1143859)

Dieng, A. B., Ruiz, F. J., & Blei, D. M. (2019). The dynamic embedded topic model. arXiv preprint arXiv:1907.05545. [Arxiv link](https://arxiv.org/abs/1907.05545)

Dieng, A. B., Ruiz, F. J., & Blei, D. M. (2020). Topic modeling in embedding spaces. Transactions of the Association for Computational Linguistics, 8, 439-453. [ACM Anthology](https://aclanthology.org/2020.tacl-1.29/),  [Arxiv link](https://arxiv.org/abs/1907.04907)